# Video Classification

This script consists all the steps for video classification.

1. extracting the images from the nii files.
2. converting the images of the nii file into the video.
3. converting the video into images again.
4. running extract_features.py to get the features from the vgg.
5. running train.py to train the deep neural model.



In [ ]:
import os
import numpy as np
import nibabel as nib # to read nii files
import shutil # for file operations
import glob  # use to make file list from diectory
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import cv2 # opencv library
import matplotlib.image as mpimg
import random
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from skimage import filters
from skimage import measure
from skimage.color import rgb2gray
from skimage.morphology import closing, square
import traceback
import sys

In [ ]:
# Declaring the home variables that will be used throughout the script.

classes = ['Alzheimer',"MCI", 'Normal']
home_files_dir = '/home/ubuntu/Select_original_fmri/'

## Converting the nii files into images

In [ ]:
data = []
labels = []

#removing existing "images" (old data) folder from the directory '/home/ubuntu/Select_original_fmri/images'
try:
    shutil.rmtree(home_files_dir + 'images/')
except:
    print("No images folder in directory")
    pass

# making "images" (for new data) folder in the '/home/ubuntu/Select_original_fmri/images' directory
try:
    os.mkdir(home_files_dir + 'images/')
except:
    pass

# iterating /home/ubuntu/Select_original_fmri/niifiles/class_ (Alzheimer, MCI , Normal)
for class_ in classes:
    print ('working on ' + class_ + '...')
    
    for root, dir ,files in os.walk(os.path.join(home_files_dir+"niifiles/" , class_)): 
        
        # making class folder inside the '/home/ubuntu/Select_original_fmri/images' directory
        try:
            os.mkdir(home_files_dir + 'images/' + class_ + '/')
        except:
            pass
        
        # iterating through each nii file in each class in the '/home/ubuntu/Select_original_fmri/niifiles' directory
        for file_ in files:
            print 'working on ' + file_ + '...'
            
            store_path = home_files_dir + 'images/' + class_ + '/' + file_ + '/'
            
            try:
                os.mkdir(store_path)
            except:
                continue
            try:
                count = 0
                
                # extracting data from nii files
                x = nib.load(os.path.join(home_files_dir + 'niifiles/' , class_) + '/' + file_).get_data()
                
                # Including only 3 dimentions(length ,width,color) and excluding time dimention
                for i in xrange(x.shape[3]):
                    for j in xrange(x.shape[2]):
                            y = x[:, :, j, i]
                            img = Image.fromarray(y)
                            img = img.convert("RGB")
                            img = img.resize([224, 224])
                            
                            #Saving image in '/home/ubuntu/Select_original_fmri/image/class_/file'
                            img.save(store_path + 'i_' + str(count) + ".jpg")
                            count+=1
            except:
                print file_
    
    #os.system('rm -rf %s%s' %(home_files_dir2, class_))

# Converting Images into Video

In [4]:
videoclassification_dir="/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/"
try:
    # removing the completed directory in the videoclassification/data directory.
    shutil.rmtree(videoclassification_dir+"completed")
except:
    print('no completed directory in data directory')
    pass

try:
    # Making "videos" directory in vieoclassification/data
    os.mkdir(videoclassification_dir+"videos/")
except:
    shutil.rmtree(videoclassification_dir+"videos/")
    os.mkdir(videoclassification_dir+"videos/")

for class_ in classes:
    try:
        # making classes(Alzheimer,MCI, Normal) inside videos folder   
        os.mkdir(os.path.join(videoclassification_dir+"videos/" , class_))
    except:
        shutil.rmtree(os.path.join(videoclassification_dir+"videos" , class_))
        os.mkdir(os.path.join(videoclassification_dir+"videos/" , class_))
    
    # Extracting images from '/home/ubuntu/Select_original_fmri/images/class_'    
    for root, dirs ,files in os.walk(os.path.join(home_files_dir + "images/" , class_)):
        for dir_ in dirs:
            print 'working on ' + dir_ + "..." + class_
            # saving videos
            save_dir = videoclassification_dir+'videos/' + class_ + '/' + dir_ + '.avi'
            print os.system("avconv  -i " + home_files_dir + 'images/' + class_ + '/' + dir_ + '/i_%d.jpg' + " -c:v libx264 " + save_dir)
try:
    os.mkdir(videoclassification_dir+"completed")
except:
    pass

working on A95_130_S_4641.nii...Alzheimer
0
working on A72_130_S_5059.nii...Alzheimer
0
working on A104_130_S_4971.nii...Alzheimer
0
working on A11_130_S_4730.nii...Alzheimer
0
working on A18_031_S_4024.nii...Alzheimer
0
working on A45_006_S_4192.nii...Alzheimer
0
working on A6_130_S_4997.nii...Alzheimer
0
working on A23_019_S_4252.nii...Alzheimer
0
working on A2_131_S_5138.nii...Alzheimer
0
working on A24_018_S_5240.nii...Alzheimer
0
working on M185_013_S_4791.nii...MCI
0
working on M144_012_S_4128.nii...MCI
0
working on M162_130_S_4883.nii...MCI
0
working on M7_006_S_4515.nii...MCI
0
working on M52_002_S_0729.nii...MCI
0
working on M180_130_S_4605.nii...MCI
0
working on M27_013_S_2324.nii...MCI
0
working on M1_013_S_2389.nii...MCI
0
working on M242_130_S_4417.nii...MCI
0
working on M43_018_S_2180.nii...MCI
0
working on M280_130_S_4415.nii...MCI
0
working on M18_002_S_4229.nii...MCI
0
working on M332_100_S_4556.nii...MCI
0
working on M14_002_S_4473.nii...MCI
0
working on M301_031_S_49

In [6]:
path = "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/videos/"
dst = "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data/"
files_list = [] 
test_list = []
train_list = []

count  = 0

try:
    os.mkdir(dst+"test/")
    os.mkdir(dst+"train/")
except:
    shutil.rmtree(dst+"test/")
    shutil.rmtree(dst+"train/")
    os.mkdir(dst+"test/")
    os.mkdir(dst+"train/")
    
for class_ in classes:
    print class_
    
    # dividing the data for training and testing purpose
    try:
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
    except:
        shutil.rmtree(dst+"train/"+class_)
        shutil.rmtree(dst+"test/"+class_)
        os.mkdir(dst+"test/"+class_)
        os.mkdir(dst+"train/"+class_)
        
    for root,dirs,files in os.walk(os.path.join(path,class_)):
        for f in files:
            files_list.append(f)
        test = int(len(files_list)*.3) # 30% test data
        train = int(round(len(files_list)*.7)) # 70% test data
        test_list += list(files_list[:test])
        train_list += list(files_list[test:train])
        
        #moving data into test and train folders
        for i in test_list:
            shutil.move(src=path+class_ +"/"+i, dst=dst+"test/"+class_+"/"+i)
        for i in train_list:
            shutil.move(src=path +  class_ +"/"+ i, dst=dst+"train/" +class_+"/"+i)
        files_list = []
        train_list = []
        test_list  = []


Alzheimer
MCI
Normal


In [13]:
% cd data

/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data


In [14]:
try:
    os.mkdir("sequences/")
    os.mkdir("checkpoints/")
except:
    shutil.rmtree("sequences/")
    shutil.rmtree("checkpoints/")
    os.mkdir("sequences/")
    os.mkdir("checkpoints/")

In [39]:
#this is used to extract images
!python 2_extract_files.py


('class_folders', ['./train/Alzheimer', './train/Normal', './train/MCI'])
('vid_class:', './train/Alzheimer')
('video path', './train/Alzheimer/A6_130_S_4997.nii.avi')
('train_or_test:', 'train')
('classname:', 'Alzheimer')
('filename_no_ext:', 'A6_130_S_4997')
('filename:', 'A6_130_S_4997.nii.avi')
Generated 6720 frames for A6_130_S_4997
('video path', './train/Alzheimer/A45_006_S_4192.nii.avi')
('train_or_test:', 'train')
('classname:', 'Alzheimer')
('filename_no_ext:', 'A45_006_S_4192')
('filename:', 'A45_006_S_4192.nii.avi')
Generated 6720 frames for A45_006_S_4192
('vid_class:', './train/Normal')
('video path', './train/Normal/N119_012_S_4026.nii.avi')
('train_or_test:', 'train')
('classname:', 'Normal')
('filename_no_ext:', 'N119_012_S_4026')
('filename:', 'N119_012_S_4026.nii.avi')
Generated 6720 frames for N119_012_S_4026
('video path', './train/Normal/N132_013_S_4580.nii.avi')
('train_or_test:', 'train')
('classname:', 'Normal')
('filename_no_ext:', 'N132_013_S_4580')
('filena

In [10]:
% cd ..

/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification


In [7]:
!python extract_features.py

Using TensorFlow backend.
Traceback (most recent call last):
  File "extract_features.py", line 16, in <module>
    from data import DataSet
  File "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/data.py", line 12, in <module>
    from processor import process_image
  File "/home/ubuntu/final_src/DeepNeuralnets--Alzheimer/videoclassification/processor.py", line 4, in <module>
    from keras.preprocessing.image import img_to_array, load_img
  File "/usr/local/lib/python2.7/dist-packages/keras/__init__.py", line 3, in <module>
    from . import utils
  File "/usr/local/lib/python2.7/dist-packages/keras/utils/__init__.py", line 6, in <module>
    from . import conv_utils
  File "/usr/local/lib/python2.7/dist-packages/keras/utils/conv_utils.py", line 3, in <module>
    from .. import backend as K
  File "/usr/local/lib/python2.7/dist-packages/keras/backend/__init__.py", line 83, in <module>
    from .tensorflow_backend import *
  File "/usr/local/lib/python2.7/dist-pa

In [ ]:
!python train.py

\begin\{*equation}

\end\{*equation}